# Text Summarization

**Summarization** can be defined as a task of producing a concise and fluent summary while preserving key information and overall meaning.

**Below is the code flow to generate summarize text:-**

Input article → split into sentences → remove stop words → build a similarity matrix → generate rank based on matrix → pick top N sentences for summary.

# 1. Import all necessary libraries

In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

# 2. Collecting reviews of a particular topic

In [2]:
df = pd.read_excel('neg_reviews.xlsx')

In [3]:
df['review'][34]

'Does the Delta Reserve cover lost items?\nIt looks like the higher end cards all use the same terms: https://www.americanexpress.com/content/dam/amex/us/credit-cards/features-benefits/policies/pdf/NAC_PLS_and_30_Updates/PP_Benefit-Guide_Tier-3-Rev-03-18.pdf\nAnd inside it says:\nEXCLUSIONS\nBenefits are not payable if the loss for which coverageissoughtwasdirectlyorindirectly,whollyor partially, contributed to or caused by:\n17. purchases lost or misplaced (lost is only coverable for Platinum and Centurion Cardmembers, this exclusion is removed for Platinum and Centurion\nCardmembers).'

In [4]:
sentences = []
for sent in df['review']:
    sentences.append(sent.replace('\n', ''))
print(len(sentences))

298


In [5]:
df['review'] = sentences

In [6]:
df['review'][34]

'Does the Delta Reserve cover lost items?It looks like the higher end cards all use the same terms: https://www.americanexpress.com/content/dam/amex/us/credit-cards/features-benefits/policies/pdf/NAC_PLS_and_30_Updates/PP_Benefit-Guide_Tier-3-Rev-03-18.pdfAnd inside it says:EXCLUSIONSBenefits are not payable if the loss for which coverageissoughtwasdirectlyorindirectly,whollyor partially, contributed to or caused by:17. purchases lost or misplaced (lost is only coverable for Platinum and Centurion Cardmembers, this exclusion is removed for Platinum and CenturionCardmembers).'

In [7]:
word_review = df[df['review'].str.contains("refund")]['review']

In [8]:
word_review = ','.join(word_review)

In [9]:
#word_review

# 3. Saving all reviews as a text document

In [10]:
file = open("refund.txt","a", encoding="utf-8" )
file.write(word_review)
file.close()

In [11]:
file = open("refund.txt", "r", encoding="utf-8")
filedata = file.readlines()
#print(filedata[0])

# 4. Generate clean sentences

In [12]:
def read_article(file_name):
    file = open(file_name, "r", encoding="utf-8")
    filedata = file.readlines()
    #print(filedata[1])
    article = filedata[0].split(". ")
    sentences = []
    for sentence in article:
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))  
    return sentences

# 5. sentence similarity function

In [13]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

# 6. Similarity matrix

**This is where we will be using cosine similarity to find similarity between sentences.**

In [14]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    return similarity_matrix

# 7. Generate Summary Method

In [37]:
def generate_summary(file_name, top_n=10):
    
    stop_words = stopwords.words('english')
    
    summarize_text = []
    
    # Step 1 - Read text and tokenize
    sentences =  read_article(file_name)
    #print(len(sentences))
    
    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)
    
    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    sentence_similarity_graph.graph
    scores = nx.pagerank(sentence_similarity_graph)
    
    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    #print("Indexes of top ranked_sentence order are ", ranked_sentence)
    
    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))
        
    # Step 5 - Offcourse, output the summarize text
    print("Summarize Text: \n", ". ".join(summarize_text))


In [38]:
generate_summary( 'refund.txt', 10 )


C:\Users\deepa\Anaconda3\lib\site-packages\nltk\cluster\util.py:131: RuntimeWarning: invalid value encountered in true_divide
  return 1 - (numpy.dot(u, v) / (sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))


PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')